In [1]:
import os
import re
import sys
import json
import time
from tqdm.notebook import tqdm
from datasets import load_dataset

In [2]:
import torch
import random
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

## Load Data

In [3]:
paper_data = load_dataset('metamong1/summarization_paper', 
    use_auth_token='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')

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
paper_data

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 73640
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 18411
    })
})

In [5]:
train_data = paper_data['train']

## Extract Data

In [6]:
train_docs = [data['text'] for data in train_data]
train_titles = [data['title'] for data in train_data]

In [63]:
target_comp = re.compile('[a-zA-Zぁ-ゔァ-ヴー々〆〤一-龥]')

In [166]:
def check_title(title, comp) :
    org_chars = re.sub('\s+', '' , title) 
    target_chars = comp.findall(org_chars)

    char_rate = len(target_chars) / len(org_chars)

    return True if char_rate >= 0.1 else False

In [120]:
sep_titles = [title for title in tqdm(train_titles) if check_title(title, target_comp)]
print('Size of data which has to be checked : %d' %len(sep_titles))

  0%|          | 0/73640 [00:00<?, ?it/s]

Size of data which has to be checked : 3722


In [167]:
check_title('대눌 노상익의 망명 의의 연구 ―독립유공자 지정과 관련하여― (大訥', target_comp)

False

## Selenium

In [19]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [20]:
chrome_path = '/Users/sanghapark/Desktop/Project/chromedriver'
driver = webdriver.Chrome(chrome_path)

<ipython-input-20-8d257ee82e31>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [21]:
title = '戰國楚地文獻 의문대체사 ‘害’,‘曷’의 의미고찰 -『清華簡 · 一』의 두 자형을 중심으로-'

In [30]:
driver.implicitly_wait(1.5)
driver.get('https://academic.naver.com/')

In [31]:
time.sleep(0.5) ## 0.5초

element = driver.find_element_by_name('query')
element.clear()

element.send_keys(title)
element.submit()

<ipython-input-31-d6b45ebb4143>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_name('query')


In [32]:
cur_url = driver.current_url

response = requests.get(cur_url)
bs = BeautifulSoup(response.content, 'html.parser')

In [33]:
div_info = bs.findAll('div', {'class' , 'ui_listing_info'})
div_info[0].find('a').text

'전국초지문헌 의문대체사 ‘해’,‘갈’의 의미고찰 -『청화간 · 일』의 두 자형을 중심으로- (戰國楚地文獻 의문대체사 ‘害’,‘曷’의 의미고찰 -『清華簡 ·...'

## Beautiful Soup

In [34]:
import requests
from bs4 import BeautifulSoup

In [130]:
title4change = [title for title in sep_titles \
    if '(' in title and ')' not in title]

print('Size : %d' %len(title4change))

Size : 174


In [69]:
base_url = 'https://academic.naver.com/search.naver?field=0&docType=1&query='

In [135]:
sep_titles[200]

'BMI(Body Mass Index)가 소득에 미치는 영향'

In [207]:
target = target_titles[17]#'황제내경 오행론에 나타난 시 관념과 의학적 정체성 ( 황제내경 오행론에'
print(target)

target_url = base_url + target
response = requests.get(target_url)
bs = BeautifulSoup(response.content, 'html.parser')
div_list = bs.find_all('div', {'class' : 'ui_listing_info'})
print(div_list[0].find('a', {'class' : 'ui_listing_subtit'}).text)

유진오의 일본어 문학에 나타난 조선적 가치의 재고 ―일본어 소설 「남곡선생」을 중심으로― (유진오의 일본어 문학에 나타난 조선적 가치의 재고 ―일본어 소설...
유진오의 일본어 문학에 나타난 조선적 가치의 재고 ―일본어 소설 「남곡선생」을 중심으로― (유진오의 일본어 문학에 나타난 조선적 가치의 재고 ―일본어 소설 「南谷先生」을...


## Naver API

In [14]:
import re
import json
import urllib.request

In [28]:
def extract_title(content) :
    content = json.loads(content)
    title = content['items'][0]['title']

    title = re.sub('<b>', ' ', title)
    title = re.sub('</b>', ' ', title)
    title = re.sub('\s+' , ' ', title).strip()
    return content

In [16]:
client_id = "FcfC2Ls3KNm5z_Zd9OG_"
client_secret = "uV2UxplHq9"

In [24]:
title = '조선 초기 원경왕후 상장례 의식과 그 특징 ( 조선 초기 원경왕후 喪葬禮'

In [25]:
encText = urllib.parse.quote(title)
url = "https://openapi.naver.com/v1/search/doc?query=" + encText # json 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if (rescode == 200) :
    response_body = response.read()
    response_content = response_body.decode('utf-8')

    org_name = extract_title(response_content)

In [29]:
org_name = extract_title(response_content)

In [30]:
org_name

{'lastBuildDate': 'Sun, 05 Dec 2021 22:20:58 +0900',
 'total': 2,
 'start': 1,
 'display': 2,
 'items': [{'title': '<b>조선 초기 원경왕후 상장례 의식과 그 특징 (조선 초기 원경왕후 喪葬禮</b> ',
   'link': 'http://academic.naver.com/article.naver?doc_id=290170169',
   'description': 'The Funeral Rites for Queen Wongyeong during Early Joseon and the Characteristics Thereof 본 연구는 <b>조선 초기</b> 凶禮의 정비 '},
  {'title': '<b>조선 초기 원경왕후 상장례 의식과 그 특징 (조선 초기 원경왕후 喪葬禮</b> ',
   'link': 'http://academic.naver.com/article.naver?doc_id=871226545',
   'description': ' 본 연구는 <b>조선 초기</b> 凶禮의 정비 과정이라는 연속선상에서 <b>원경왕후</b> 국상을 하나의 기점으로 파악'}]}